0. есть датафрейм df, колонки "title" - название статьи, "annotation" - аннотация, "text" - текст статьи, "authors" - авторы
1. Векторизация текстов с учётом семантических особенностей
2. Кластеризация (нужны небольшие кластеры)
3. Тестирование: выводим статью, смотрим на те же, что в кластере.

SentenceTransformer - работает с предложениями

In [109]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from tqdm import tqdm
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

In [31]:
df = pd.read_excel('date_it_lemm.xlsx')

In [33]:
tqdm.pandas() #применяем прогресс-бар для пандаса

### 1. SentenceTransformer

In [35]:
model = SentenceTransformer('all-MiniLM-L6-v2') #предобученная модель

df['text_embedding'] = df['text'].progress_apply(lambda x: model.encode(x)) #векторизация

100%|██████████| 2308/2308 [01:55<00:00, 20.06it/s]


In [40]:
embeddings = np.array(df['text_embedding'].tolist()) #эмбеддинги в массив

In [56]:
#диапазон кластеров здесь напишу чтобы удобно было менять, а не в коде
min_cluster_size = 3
max_cluster_size = 20

#### 1.1 Agglomerative clustering

In [50]:
clustering = AgglomerativeClustering(n_clusters=n_clusters,linkage='average')
df['cluster'] = clustering.fit_predict(embeddings)

In [54]:
cluster_sizes = df['cluster'].value_counts()
cluster_sizes

cluster
28     55
52     36
45     28
366    26
83     26
       ..
389     1
501     1
692     1
444     1
244     1
Name: count, Length: 769, dtype: int64

In [67]:
import random

random_index = random.randint(0, len(df) - 1) #случайно статью выберем
random_article = df.iloc[random_index]

print(f"Title: {random_article['title']}")
print(f"Authors: {random_article['authors']}")
print(f"Annotation: {random_article['annotation']}")
print(f"Text: {random_article['text'][:500]}...")  # Выводим первые 500 символов текста

cluster_id = random_article['cluster'] #статьи из того же кластера
cluster_articles = df[df['cluster'] == cluster_id]

print("\nСтатьи из того же кластера:")
for _, article in cluster_articles.iterrows():
    print(f"Title: {article['title']}, Authors: {article['authors']}")

Title: ПРОЦЕДУРЫ ПРИМЕНЕНИЯ АДАПТИВНЫХ СИТУАЦИОННЫХ ТРЕНАЖЕРОВ
Authors: Удальцов Николай Петрович, Агеев Павел Александрович, Михейкина Елена Викторовна
Annotation: В статье рассматриваются основные процедуры применения адаптивных ситуационных тренажеров на основе моделирования деятельности должностных лиц органов управления с целью совершенствования способов и методов работы и повышения качества управления.
Text: процедура применение адаптивный ситуационный тренажёр н п удальцов п агеев михейкина статья рассматриваться основной процедура применение адаптивный ситуационный тренажёр основа моделирование деятельность должностной лицо орган управление цель совершенствование способ метод работа повышение качество управление ключевой слово должностной лицо дежурный смена орган пункт управление тренажёр моделирование обучение подготовка должностной лицо орган управление различный уровень кропотливый сложный про...

Статьи из того же кластера:
Title: Моделирование в психологических, социальны

Вообще не похоже. Попробуем другие алгоритмы кластеризации.

#### 1.2 DBSCAN

In [99]:
eps = 0.07 #максимальное расстояние между двумя образцами для того, чтобы они считались соседями
min_samples = 2  #минимальное количество образцов в окрестности точки, чтобы она считалась ядром

clustering = DBSCAN(eps=eps, min_samples=min_samples, metric='cosine')
df['cluster'] = clustering.fit_predict(embeddings)

cluster_sizes = df['cluster'].value_counts()
print("Размеры кластеров:")
print(cluster_sizes)

if -1 in cluster_sizes:
    print(f"Количество статей, не попавших в кластеры (шум): {cluster_sizes[-1]}")

Размеры кластеров:
cluster
 0     1352
-1      798
 9       17
 22      10
 3        6
       ... 
 24       2
 27       2
 28       2
 29       2
 61       2
Name: count, Length: 63, dtype: int64
Количество статей, не попавших в кластеры (шум): 798


В DBSCAN как ни крути параметры, всё равно либо очень много шума, либо очень много статей в одном кластере. В плане сглаженности размеров кластеров аггломеративная лучше была.

#### 1.3 K-means

In [120]:
n_clusters = len(df) // 8
print(f"Количество кластеров: {n_clusters}")

Количество кластеров: 288


In [130]:
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df['cluster'] = kmeans.fit_predict(embeddings)

cluster_sizes = df['cluster'].value_counts()
print("Размеры кластеров:")
print(cluster_sizes)

cluster_sizes = df['cluster'].value_counts()

single_item_clusters = (cluster_sizes == 1).sum()

# Выводим результат
print(f"Количество кластеров с ровно 1 элементом: {single_item_clusters}")

Размеры кластеров:
cluster
195    27
137    25
178    24
131    22
63     22
       ..
286     1
250     1
201     1
129     1
231     1
Name: count, Length: 288, dtype: int64
Количество кластеров с ровно 1 элементом: 20


In [136]:
random_index = random.randint(0, len(df) - 1)
random_article = df.iloc[random_index]

print(f"Title: {random_article['title']}")
print(f"Authors: {random_article['authors']}")
print(f"Annotation: {random_article['annotation']}")
print(f"Text: {random_article['text'][:500]}...")  # Выводим первые 500 символов текста

cluster_id = random_article['cluster']
cluster_articles = df[df['cluster'] == cluster_id]

print("\nСтатьи из того же кластера:")
for _, article in cluster_articles.iterrows():
    print(f"Title: {article['title']}, Authors: {article['authors']}")

Title: МОДЕЛИРОВАНИЕ РАБОТЫ СЕРВИСА БЫСТРОГО РЕАГИРОВАНИЯ ДЛЯ ОБЕСПЕЧЕНИЯ РАБОТЫ ТЕРМИНАЛЬНО-СКЛАДСКИХ КОМПЛЕКСОВ
Authors: Упырь Роман Юрьевич, Дудакова Анастасия Владимировна
Annotation: Сервис быстрого реагирования-современный тренд во многих областях деятельности, в том числе и на железнодорожном транспорте. С помощью выездных мобильных бригад проводят техническое обслуживание вагонов, их коммерческий осмотр, погрузочно-разгрузочные операции, оформление документов и др. В статье...
Text: р ю упырь дудаков иркутский государственный университет путь сообщение иргупс г иркутск российский федерация моделирование работа сервис быстрый реагирование обеспечение работа терминальный складской комплекс аннотагщть сервис быстрый реагирование современный тренд многий область деятельность число железнодорожный транспорт помощть выездной мобильный бригада проводить технический обслуживание вагон коммерческий осмотр погрузочный разгрузочный операция оформление документ др статья пример восточн...


И снова смысла достаточно мало.

### 2. FastText

In [143]:
import gensim.downloader as api
from gensim.models import FastText

In [172]:
model = FastText.load_fasttext_format('cc.ru.300.bin')

C:\Users\ksurashanti\AppData\Local\Temp\ipykernel_7148\1782492091.py:1: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  model = FastText.load_fasttext_format('cc.ru.300.bin')


In [196]:
def vectorize_text(text):
    words = text.split()
    word_vectors = [model.wv(word) for word in words]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(model.get_dimension())

In [194]:
help(FastText)

Help on class FastText in module gensim.models.fasttext:

class FastText(gensim.models.word2vec.Word2Vec)
 |  FastText(sentences=None, corpus_file=None, sg=0, hs=0, vector_size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, word_ngrams=1, sample=0.001, seed=1, workers=3, min_alpha=0.0001, negative=5, ns_exponent=0.75, cbow_mean=1, hashfxn=<built-in function hash>, epochs=5, null_word=0, min_n=3, max_n=6, sorted_vocab=1, bucket=2000000, trim_rule=None, batch_words=10000, callbacks=(), max_final_vocab=None, shrink_windows=True)
 |
 |  Method resolution order:
 |      FastText
 |      gensim.models.word2vec.Word2Vec
 |      gensim.utils.SaveLoad
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __init__(self, sentences=None, corpus_file=None, sg=0, hs=0, vector_size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, word_ngrams=1, sample=0.001, seed=1, workers=3, min_alpha=0.0001, negative=5, ns_exponent=0.75, cbow_mean=1, hashfxn=<built-in function has

In [198]:
df['text_vector'] = df['text'].progress_apply(vectorize_text)

  0%|          | 1/2308 [00:00<?, ?it/s]


TypeError: 'FastTextKeyedVectors' object is not callable